In [1]:
import os

In [2]:
print("../")

../


In [3]:
%pwd

'f:\\end_to_end_flight_price_prediction\\FLIGHT_PRICE_PREDICTION\\research'

In [4]:
from dataclasses import dataclass  # the dataclass decorator for simplifying class creation
from pathlib import Path  # Path for handling file system paths 

@dataclass(frozen=True)  #  simplifies the class structure; frozen makes it immutable
class DataIngestionConfig:
    root_dir: Path  # Root directory for managing all data ingestion-related files and outputs
    source_URL: str  # URL of the data source to be downloaded
    local_data_file: Path  # Path where the downloaded data file will be stored locally
    unzip_dir: Path  # Directory where the data will be extracted after downloading


In [1]:
from Flight_Price.constants import *
from Flight_Price.utils.common import read_yaml,create_directories

In [ ]:
from Flight_Price.constants import *
from Flight_Price.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,  # Path to the main config file
        params_filepath = PARAMS_FILE_PATH,  # Path to the parameters file
        schema_filepath = SCHEMA_FILE_PATH  # Path to the schema file
    ):
        self.config = read_yaml(config_filepath)  # Load configuration settings
        self.params = read_yaml(params_filepath)  # Load parameter settings
        self.schema = read_yaml(schema_filepath)  # Load schema details

        create_directories([self.config.artifacts_root])  # Ensure artifact root directory exists

        # Artifacts setup starts here for data ingestion

    def get_data_ingestion_config(self) -> DataIngestionConfig:  # Returns data ingestion configuration
        config = self.config.data_ingestion  # Access data ingestion settings

        create_directories([config.root_dir])  # Ensure the root directory for data ingestion exists

        # Populate and return the DataIngestionConfig object with relevant paths and URLs
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

NameError: name 'CONFIG_FILE_PATH' is not defined